In [22]:
# Import numpy and pandas library
import pandas as pd
import numpy as np

#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

# Input training and test data
deliveries = pd.read_csv("Data/deliveries.csv")
match = pd.read_csv("Data/matches.csv")
IPL_2018 = pd.read_csv("Data/test_data.csv")
# Choice of fields for current model from available data
matches = match[["city","venue","season","toss_winner","toss_decision", "result", "dl_applied", "team1", "team2", "winner"]]


In [23]:
# Addition of customized field
match_by_ball = deliveries[["match_id", "inning", "batsman","batsman_runs", "extra_runs", "total_runs"]]

match_by_ball = match_by_ball[match_by_ball["inning"] == 1]
match_by_ball.drop(["inning"], axis=1, inplace=True)

extra_stats = match_by_ball.groupby(["match_id", "batsman"])["batsman_runs", "extra_runs", "total_runs"].sum()

extra_stats["half_century"] = extra_stats["batsman_runs"] >= 50

newdf = extra_stats.groupby("match_id").sum().drop("batsman_runs",1).reset_index(drop = True)
matches = pd.concat([matches, newdf], axis=1)

In [24]:
# Choice of fields for test data
IPL_2018 = IPL_2018[["city","venue","season","toss_winner","toss_decision", "result", "dl_applied", "team1", "team2", "winner", "total_runs", "half_century", "extra_runs"]]

In [25]:
# Removing matches resulting in tie, having no results or being finalized via D/L method
matches = matches[matches["result"] != "tie"]
matches = matches[matches["result"] != "no result"]
matches = matches[matches["dl_applied"] == 0]
IPL_2018 = IPL_2018[matches["result"] != "tie"]
IPL_2018 = IPL_2018[matches["result"] != "no result"]
IPL_2018 = IPL_2018[matches["dl_applied"] == 0]

/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [26]:
# Merging training and test data for preprocessing
matches = pd.concat([IPL_2018, matches]).reset_index(drop=True)

matches.drop(["dl_applied", "result"],1,inplace=True)

In [27]:
def factorize_fields(matches):
    df = matches[["team1","team2","toss_winner","winner"]]
    _, b = pd.factorize(df.values.T.reshape(-1, ))  
    facorized_fields = df.apply(lambda x: pd.Categorical(x, b).codes)
    matches["venue"] = matches.venue.factorize()[0]
    matches["city"] = matches.city.factorize()[0]
    matches["toss_winner"] = matches.toss_winner.factorize()[0]
    matches["toss_decision"] = matches.toss_decision.factorize()[0]
    matches = pd.concat([matches.drop(["team1","team2","toss_winner","winner"],1), facorized_fields], 1)
    factors = pd.factorize(df.values.T.reshape(-1, ))[1]
    return [factors, matches]
#[["city","venue","season","toss_winner","toss_decision"]]

In [28]:
# Converting data into factors
[factors, matches] = factorize_fields(matches)

# Removing season: Include if time model made
matches.drop(["season"],1,inplace=True)

In [29]:
# Deriving insight on data
matches.describe()

,city,extra_runs,half_century,toss_decision,total_runs,venue,team1,team2,toss_winner,winner
count,619.000000,619.00000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000,619.000000
mean,8.109855,8.49273,0.799677,0.424879,160.460420,12.684976,4.615509,4.957997,4.768982,4.408724
std,7.638106,4.36645,0.738625,0.494724,29.675477,9.985229,3.628477,3.641479,3.652805,3.444331
min,-1.000000,0.00000,0.000000,0.000000,67.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.500000,5.00000,0.000000,0.000000,141.000000,3.000000,1.000000,2.000000,2.000000,1.000000
50%,5.000000,8.00000,1.000000,0.000000,162.000000,10.000000,4.000000,5.000000,4.000000,4.000000
75%,11.000000,11.00000,1.000000,1.000000,181.000000,17.000000,8.000000,8.000000,8.000000,8.000000
max,30.000000,27.00000,3.000000,1.000000,263.000000,37.000000,13.000000,13.000000,13.000000,13.000000


In [30]:
# Train - test split after pre processing
ind = len(IPL_2018)
test_data = matches[:ind]
train_data = matches[ind:]

In [31]:
def remap(item):
    '''Function to remap the factors to original field names'''
    return factors[item]

In [32]:
def model_building(model, predictors, outcome, data, test_data):
    '''Function to build model, cross-validate and predict results'''
    #model.fit(data[predictors], data[outcome])  
    kf = KFold(data.shape[0], n_folds = 3)
    error = []
    for train, test in kf:
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    cv_error = np.mean(error)
    #print('Cross validation Score : %s' % '{0:.3%}'.format(cv_error))
    model.fit(data[predictors],data[outcome])
    #coefficients = [model.intercept_, model.coef_]
    #print coefficients
    predictions = model.predict(test_data[predictors])
    test_data["predicted_winner"] = predictions
    accuracy = metrics.accuracy_score(predictions,test_data[outcome])
    #print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    test_data["Team1"] = test_data["team1"].apply(remap)
    test_data["Team2"] = test_data["team2"].apply(remap)
    test_data["Actual Winner"] = test_data["winner"].apply(remap)
    test_data["Predicted Winner"] = test_data["predicted_winner"].apply(remap)
    df = test_data[["Team1","Team2","Actual Winner", "Predicted Winner"]]
    return [df, accuracy, cv_error]

In [33]:
# Models tested
model1 = RandomForestClassifier(n_estimators=100)
model2 = LogisticRegression()         #L2 regularization, one vs all
model3 = LogisticRegression(penalty='l1')         #L1 regularization, one vs all
model4 = LogisticRegression(solver='newton-cg', multi_class='multinomial')  #Multinomial
model5 = SVC(kernel = "linear")
models = [model1, model2, model3, model4, model5]
results = []
accuracies = []
cv_errors = []
#col = ["Intercept"] + list(train_data.columns)
#coefficients_summary = pd.DataFrame(columns= col)
output = ['winner']
predictors = matches.drop(["winner"],1).columns
for model in models:
    [result, accuracy, cv_error] = model_building(model, predictors, output, train_data, test_data)
    results.append(result)
    accuracies.append(accuracy)
    cv_errors.append(cv_error)
    #coefficients_summary = coefficients_summary.append(coefficients, ignore_index=True)
model_names = ["Random Forest", "LogisticRegression(One vs All) L2 reg", "LogisticRegression(One vs All) L1 reg", "MultinomialRegression", "SVM"]
model_comparison = pd.DataFrame(columns=["Model Names", "Accuracy", "Cross Validation Errors"])
model_comparison["Model Names"] = model_names
model_comparison["Accuracy"] = accuracies
model_comparison["Cross Validation Errors"] = cv_errors


/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_l

In [34]:
# Final result
model_comparison

,Model Names,Accuracy,Cross Validation Errors
0,Random Forest,0.333333,0.522997
1,LogisticRegression(One vs All) L2 reg,0.111111,0.216395
2,LogisticRegression(One vs All) L1 reg,0.111111,0.229499
3,MultinomialRegression,0.111111,0.221345
4,SVM,0.000000,0.244301


In [35]:
# Post model fit analysis
import statsmodels.formula.api as sm
model = sm.MNLogit(train_data[output], train_data[predictors]) 
mod = model.fit()

         Current function value: 1.914973
         Iterations: 35


/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [36]:
print mod.summary()

                          MNLogit Regression Results                          
Dep. Variable:                 winner   No. Observations:                  610
Model:                        MNLogit   Df Residuals:                      493
Method:                           MLE   Df Model:                          104
Date:                Tue, 17 Apr 2018   Pseudo R-squ.:                  0.1891
Time:                        19:36:48   Log-Likelihood:                -1168.1
converged:                      False   LL-Null:                       -1440.6
                                        LLR p-value:                 5.933e-61
     winner=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
city              0.1686      0.044      3.875      0.000       0.083       0.254
extra_runs       -0.0210      0.037     -0.566      0.572      -0.094       0.052
half_century     -0.0633      0.279     